In [17]:
import numpy as np
import pandas as pd


path = '/opt/ml/input/data/'

train = pd.read_csv(path + 'train_data.csv')
test = pd.read_csv(path + 'test_data.csv')

In [26]:
tem = pd.to_datetime(train['Timestamp'])
tem.dt.year # 연도 정보
tem.dt.month # 월 정보
tem.dt.day # 일 정보
tem.dt.hour # 시간 정보
tem.dt.minute # 분 정보
tem.dt.second # 초 정보
train['year'] = tem.dt.year
train['year'].value_counts()

2020    2266567
2019         19
Name: year, dtype: int64

In [3]:
train = train.sort_values(by=["userID", "Timestamp"], axis=0)
train = train[["userID", "assessmentItemID", "testId", "answerCode", "KnowledgeTag"]]
train = train.groupby("userID").apply(
                lambda r: (
                    r["testId"].values,
                    r["assessmentItemID"].values,
                    r["KnowledgeTag"].values,
                    r["answerCode"].values,
                )
            )

In [16]:
train

userID
0       ([A060000001, A060000001, A060000001, A0600000...
1       ([A040000013, A040000013, A040000013, A0400000...
2       ([A030000050, A030000050, A030000050, A0300000...
5       ([A080000001, A080000001, A080000001, A0800000...
6       ([A030000016, A030000016, A030000016, A0300000...
                              ...                        
7436    ([A050000095, A050000095, A050000095, A0500000...
7437    ([A040000072, A040000072, A040000072, A0400000...
7438    ([A080000002, A080000002, A080000002, A0800000...
7440    ([A050000096, A050000096, A050000096, A0500000...
7441    ([A030000071, A030000071, A030000071, A0300000...
Length: 6698, dtype: object

In [39]:
print(len(set(train['userID'].unique()) - set(test['userID'].unique())))
print(len(set(train['userID'].unique())))

6698
6698


train과 test 간 같은 유저가 존재하지 않는다.

# 시험 문제별로

In [27]:
for i in range(3):
    print(f'{i}번째 value_count')
    print(train['assessmentItemID'].str[i].value_counts())
    print('')

0번째 value_count
A    2266586
Name: assessmentItemID, dtype: int64

1번째 value_count
0    2266586
Name: assessmentItemID, dtype: int64

2번째 value_count
7    279164
5    275773
3    273762
1    272082
2    268327
4    267323
6    264434
8    246336
9    119385
Name: assessmentItemID, dtype: int64



In [26]:
train['large'] = train['assessmentItemID'].str[2]
print('large 그룹 별 문제 개수')
print(train.groupby('large')['answerCode'].count())
print('large 그룹 별 맞을 확률')
print(train.groupby('large')['answerCode'].mean())

large 그룹 별 문제 개수
large
1    272082
2    268327
3    273762
4    267323
5    275773
6    264434
7    279164
8    246336
9    119385
Name: answerCode, dtype: int64
large 그룹 별 맞을 확률
large
1    0.800876
2    0.737593
3    0.702238
4    0.684056
5    0.658208
6    0.709232
7    0.521876
8    0.502598
9    0.449948
Name: answerCode, dtype: float64


대그룹별 문제 난이도가 확연히 보인다.

In [45]:
train['testID'] = train['testId'].str[-3:]
train.groupby('testID')['answerCode'].mean().sort_values()

testID
057    0.529107
047    0.542518
049    0.550103
051    0.557857
069    0.558188
         ...   
179    0.819098
185    0.831546
195    0.835409
198    0.853838
196    0.895706
Name: answerCode, Length: 198, dtype: float64

In [37]:
train['assessmentItemID'].str[4:7].value_counts()

017    17739
009    17555
011    17029
003    16970
007    16826
       ...  
180     3767
197     2973
194     2596
196     1304
198      951
Name: assessmentItemID, Length: 198, dtype: int64

6698